In [6]:
import os
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt

In [7]:
def cutmix(image1, image2, bbox1, bbox2, alpha=0.2):
    height, width, _ = image1.shape

    # Generate random bounding box
    cut_rat = np.random.rand()
    cut_w = int(width * cut_rat)
    cut_h = int(height * cut_rat)

    # Generate random center for the cut area
    cut_x = np.random.randint(width)
    cut_y = np.random.randint(height)

    # Determine the bounding box coordinates
    x1 = np.clip(cut_x - cut_w // 2, 0, width)
    y1 = np.clip(cut_y - cut_h // 2, 0, height)
    x2 = np.clip(cut_x + cut_w // 2, 0, width)
    y2 = np.clip(cut_y + cut_h // 2, 0, height)

    mixed_image = image1.copy()
    mixed_image[y1:y2, x1:x2, :]= image2[y1:y2, x1:x2, :]

    bbox1 = np.array(bbox1)
    bbox2 = np.array(bbox2)
    bbox2[:, [1, 3]] = bbox2[:, [1, 3]] * (cut_w / width) + x1 / width
    bbox2[:, [2, 4]] = bbox2[:, [2, 4]] * (cut_h / height) + y1 / height
    mixed_bboxes = np.vstack((bbox1, bbox2))

    return mixed_image, mixed_bboxes

In [8]:
def read_bboxes(label_path):
    with open(label_path, 'r') as f:
        bboxes = [line.strip().split() for line in f.readlines()]
    
    return [[int(bbox[0])] + [float(x) for x in bbox[1:]] for bbox in bboxes]

In [9]:
def save_cutmix_images_and_labels(image_dir, label_dir, output_image_dir, output_label_dir, alpha=0.2):
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)
    
    for img_file in image_files:
        img_path = os.path.join(image_dir, img_file)
        label_path = os.path.join(label_dir, img_file.rsplit('.', 1)[0] + '.txt')
        image1 = cv2.imread(img_path)
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
        bbox1 = read_bboxes(label_path)

        # Randomly select another image and label for cutmix
        img_file2 = random.choice(image_files)
        while img_file2 == img_file:
            img_file2 = random.choice(image_files)

        img_path2 = os.path.join(image_dir, img_file2)
        label_path2 = os.path.join(label_dir, img_file2.rsplit('.', 1)[0] + '.txt')
        image2 = cv2.imread(img_path2)
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
        bbox2 = read_bboxes(label_path2)

        mixed_image, mixed_bboxes = cutmix(image1, image2, bbox1, bbox2, alpha)

        mixed_image_bgr = cv2.cvtColor(mixed_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(output_image_dir, img_file), mixed_image_bgr)

        with open(os.path.join(output_label_dir, img_file.rsplit('.', 1)[0] + '.txt'), 'w') as f:
            for bbox in mixed_bboxes:
                f.write(f"{int(bbox[0])} {' '.join(map(str, bbox[1:]))}\n")
        print(f"Applied cutmix and saved: {img_file}")

In [10]:
input_path = 'my_data'
output_path = 'test_cutmix_1'
save_cutmix_images_and_labels(f'{input_path}/images/train', f'{input_path}/labels/train', f'{output_path}/images/train', f'{output_path}/labels/train', alpha=0.2)

Applied cutmix and saved: frame_100200.jpg
Applied cutmix and saved: frame_101280.jpg
Applied cutmix and saved: frame_101460.jpg
Applied cutmix and saved: frame_101520.jpg
Applied cutmix and saved: frame_101820.jpg
Applied cutmix and saved: frame_101880.jpg
Applied cutmix and saved: frame_101940.jpg
Applied cutmix and saved: frame_1020.jpg
Applied cutmix and saved: frame_102060.jpg
Applied cutmix and saved: frame_102180.jpg
Applied cutmix and saved: frame_102420.jpg
Applied cutmix and saved: frame_102480.jpg
Applied cutmix and saved: frame_103500.jpg
Applied cutmix and saved: frame_103680.jpg
Applied cutmix and saved: frame_103740.jpg
Applied cutmix and saved: frame_104580.jpg
Applied cutmix and saved: frame_104640.jpg
Applied cutmix and saved: frame_104700.jpg
Applied cutmix and saved: frame_105120.jpg
Applied cutmix and saved: frame_105180.jpg
Applied cutmix and saved: frame_106320.jpg
Applied cutmix and saved: frame_106800.jpg
Applied cutmix and saved: frame_106980.jpg
Applied cutmi